In [1]:
import pyphi
import numpy as np
import matplotlib.pyplot as plt
from scipy import optimize
import pandas as pd
from collections import defaultdict


%matplotlib inline
pyphi.__file__

'/usr/local/lib/python3.5/dist-packages/pyphi/__init__.py'

## Point to Note

IIT makes the assumption that mechanisms (nodes) are independent of each other, i.e. that the state of each node in the present is not influenced by the state of the other nodes in the present. This allows a convenient simplification.

Beforehand, the simplest way to comprehensively capture the properties of a network was with the $2^n \times 2^n$ transition probability matrix (tpm). However, this assumption allows us to reduce this matrix into a $2^n \times n$ matrix. 

Let this matrix be denoted by $\mathbf{T}$. Beforehand, $\mathbf{T}_{ij}$ would refer to the probability of transferring from state $j$ to state $i$. In the reduced tpm however, $\mathbf{T}_{ij}$ refers to the probability that the $j$th node will enter into state '1', given that the system was in state $i$. In otherwords, each column of this reduced matrix refers to the state of each node individually. If the mechanisms of the system are truly independent of each other, then the former and larger matrix can be reconstructed from this reduced matrix.

**Note**: Last week I was having difficulty with the matlab code changing my input matrix to this other form which was not the same. The problem was that the model I had defined did not satisfy independence properly. 

## IIT convention

If I have a transition probability matrix of the form:

$$
\left[\begin{matrix} a11 & a12\\
                a21 &a22\\
                a31 & a32\\
                a41 & a42
\end{matrix}\right]
$$

although the rows and columns are not labelled, we understand them to have the following meaning. In this order, the rows refer to the following states: AB = 00, AB = 10, AB = 01, AB = 11. The columns meanwhile refer to A and B read from left to right. (A and B are the names of the nodes, and it is assumed that each node has only 2 states).

# Simple 2 Nodes System

First we examine an example of an optimal network of 2 nodes. Below is the state by node transition probability matrix.

In [2]:
tpm = np.array([
...     [0, 0],
...     [0, 1],
...     [1, 1],
...     [1, 0],
... ])

Looking at this matrix, we note that it is deterministic. Futhermore, given a starting location two options exist for how the system can evolve. Either the system starts at 00 and stays there, or else it cycles between the states 10->01->11->10...

We now measure the $\Phi$ for the system given each of these starting states:

In [3]:
network = pyphi.Network(tpm)
state = (1, 0)
subsystem = pyphi.Subsystem(network, state, range(network.size))
phi = pyphi.compute.big_phi(subsystem)
print (phi)

1.5


In [4]:
state = (0,1)
subsystem = pyphi.Subsystem(network, state, range(network.size))
phi = pyphi.compute.big_phi(subsystem)
print (phi)

1.5


In [5]:
state = (1,1)
subsystem = pyphi.Subsystem(network, state, range(network.size))
phi = pyphi.compute.big_phi(subsystem)
print (phi)

1.5


In [6]:
state = (0,0)
subsystem = pyphi.Subsystem(network, state, range(network.size))
phi = pyphi.compute.big_phi(subsystem)
print (phi)

1.5


We see that all the starting states yield the same value of $\Phi$. This may be somewhat surprising given that the state 00 behaves so differently from the others.

Next we ask the question, what feature of this system causes $\Phi$ to be what it is. (This seems to be the largest possible $\Phi$ for a 2 nodes system). 

By exhaustively checking all deterministic tpm's we find the following: (The code used to determine these results can be found in appendix 1)

1. The maximum value of $\Phi$ is 1.5. This value occurs for exactly 8 tpms, and independently of initial state. All of these tpm's have the same property as the one previously shown. The set of states is split into 2 sets, containing 1 and 3 states respectively. The set of 3 states cycles as previous. It can be shown combinatorially that only 8 such tpms exist (for a system of 2 nodes).

2. Not all tpms have a value of $\Phi$ that is independent of the initial state. As of yet, no patturn has been observed as to why this is.

3. Exactly 4 tpms have $\Phi=1$, and this value is independent of initial state. Two of these tpms describe cycles through all 4 nodes, and 2 of them describe 2 sets of 2 states each, which swap with each iteration. Now it is noted that there exist 6 different full cycles, and 6 different ways to divide 4 states into 2 sets of 2. However, of the 6 possible cycles through all states, only the 2 that appear here have the property that the inidividual states of both nodes are each influenced by the other. The other cycles might have a scenario where one node flips each time to whatever state it was not in previously. This isolated node immediatly reduces $\Phi$. In other words, only 2 of these 6 cycles are irreducible. This is the same reason for the other case as well.




Thus we establish 2 properties which we can look for in a network which will influence the size of $\Phi$

1. Irreducibility. This is particularly obvious for a system of 2 nodes, and has bigger consequences. But if one node turns on and off independently of the other, then the system is reducible and $\Phi$ is 0. This limit seems to hold in the probabilistic case as a node is increasingly decoupled from the rest, $\Phi$ tends to 0.
2. Occurance of States: In all the systems which have $\Phi \geq 1$ independently of the initial state, we observe that: $\forall s \in \mathbf{S}, \quad \exists s' \in \mathbf{S}$ such that if $s'$ is the intial state, then $p(X^n = s|X^0=s')=1$ for some $n \in \mathbb{N}$. (Using the notation that $X^i$ is the state of the system after $i$ steps, $\mathbf{S}$ is the set of all possible states of the system, and assuming the system to be deterministic). Alternatively if we define $f$ to be the function which iterates the system, there does not exist a state $s$ such that the preimage $f^{-1}(s)$ is the empty set.
3. Partitioning of States: Provided that each state will occur indefinitely given particular initial states, we now have a few options. 
    - All 4 states could belong to the same set and occur in a cycle.
    - The 4 states could be split into 2 sets of 2.
    - The 4 states coudl be split into sets of 3 and 1.
    
    Without any insights as to why, we observe that the first two cases generate a $\Phi$ of $1$, while the last one has $\Phi=1.5$. It seems that the minimum number of states in one such set does not matter too much. In contrast, the max number of states in a set does seem important given the advantage of the 3,1 split over the 2,2 split. Finally, having a partition seems to be more important again given that the 3,1 split performs better than having all 4 states in one loop.

# 3 Node System

In this section, we seek to apply the principles observed and guessed at previously to find networks of 3 nodes which have a large value of $\Phi$.

Below we define a tpm which partitions all 8 states into 2 sets, containing 5 and 3 states respectively. Given an inital state in one of these sets, all future states will belong to that set.

In [7]:
tpm = np.array([
...     [0, 1,0],
...     [1, 1, 1],
...     [0, 0, 1],
...     [1, 0, 0],
...     [0, 0, 0],
...     [0, 1, 1],
...     [1, 1, 0],
...     [1, 0, 1],
... ])

In [8]:
network = pyphi.Network(tpm)
state = (1, 1,0)
subsystem = pyphi.Subsystem(network, state, range(network.size))
phi = pyphi.compute.big_phi(subsystem)
print (phi)

1.479165


And while this system consistently has a fairly large value of $\Phi$, the value is no longer independent of the initial state. We'll return to this later.

## Appendix 1

In [16]:
options = np.array([[int(x) for x in bin(i)[2:].zfill(8)] for i in range(2**8)])

eps = 1e-5
options_perturbed = np.array([np.array([(float(j)-eps if j==1 else eps) for j in i]) for i in options])

tpms = np.array([np.reshape(matrix, (4,2)) for matrix in options_perturbed])

In [97]:
PHIS1 = set([])
key1 = {}
results1 = defaultdict(list)

count = 0
for tpm in tpms:
    network = pyphi.Network(tpm)
    state = (0,0)

    subsystem = pyphi.Subsystem(network, state, range(network.size))

    phi = pyphi.compute.big_phi(subsystem)
    phi = round(phi,2)
    
    if phi not in PHIS1:
        count+=1
        PHIS1.add(phi);
        key1[phi] = count;
    
    results1[key1[phi]].append(tpm.round())

In [98]:
PHIS2 = set([])
key2 = {}
results2 = defaultdict(list)

count = 0
for tpm in tpms:
    network = pyphi.Network(tpm)
    state = (0,1)

    subsystem = pyphi.Subsystem(network, state, range(network.size))

    phi = pyphi.compute.big_phi(subsystem)
    phi = round(phi,2)
    
    if phi not in PHIS2:
        count+=1
        PHIS2.add(phi);
        key2[phi] = count;
    
    results2[key2[phi]].append(tpm.round())

In [99]:
PHIS3 = set([])
key3 = {}
results3 = defaultdict(list)

count = 0
for tpm in tpms:
    network = pyphi.Network(tpm)
    state = (1,0)

    subsystem = pyphi.Subsystem(network, state, range(network.size))

    phi = pyphi.compute.big_phi(subsystem)
    phi = round(phi,2)
    
    if phi not in PHIS3:
        count+=1
        PHIS3.add(phi);
        key3[phi] = count;
    
    results3[key3[phi]].append(tpm.round())

In [100]:
PHIS4 = set([])
key4 = {}
results4 = defaultdict(list)

count = 0
for tpm in tpms:
    network = pyphi.Network(tpm)
    state = (1,1)

    subsystem = pyphi.Subsystem(network, state, range(network.size))

    phi = pyphi.compute.big_phi(subsystem)
    phi = round(phi,2)
    
    if phi not in PHIS4:
        count+=1
        PHIS4.add(phi);
        key4[phi] = count;
    
    results4[key4[phi]].append(tpm.round())

In [118]:
for i in results1[key1[1]]:print(i)

[[ 0.  0.]
 [ 0.  1.]
 [ 1.  0.]
 [ 1.  1.]]
[[ 0.  1.]
 [ 0.  0.]
 [ 1.  1.]
 [ 1.  0.]]
[[ 1.  0.]
 [ 1.  1.]
 [ 0.  0.]
 [ 0.  1.]]
[[ 1.  1.]
 [ 1.  0.]
 [ 0.  1.]
 [ 0.  0.]]


In [119]:
for i in results2[key2[1]]:print(i)

[[ 0.  0.]
 [ 0.  1.]
 [ 1.  0.]
 [ 1.  1.]]
[[ 0.  1.]
 [ 0.  0.]
 [ 1.  1.]
 [ 1.  0.]]
[[ 1.  0.]
 [ 1.  1.]
 [ 0.  0.]
 [ 0.  1.]]
[[ 1.  1.]
 [ 1.  0.]
 [ 0.  1.]
 [ 0.  0.]]


In [120]:
for i in results3[key3[1]]:print(i)

[[ 0.  0.]
 [ 0.  1.]
 [ 1.  0.]
 [ 1.  1.]]
[[ 0.  1.]
 [ 0.  0.]
 [ 1.  1.]
 [ 1.  0.]]
[[ 1.  0.]
 [ 1.  1.]
 [ 0.  0.]
 [ 0.  1.]]
[[ 1.  1.]
 [ 1.  0.]
 [ 0.  1.]
 [ 0.  0.]]


In [121]:
for i in results4[key4[1]]:print(i)

[[ 0.  0.]
 [ 0.  1.]
 [ 1.  0.]
 [ 1.  1.]]
[[ 0.  1.]
 [ 0.  0.]
 [ 1.  1.]
 [ 1.  0.]]
[[ 1.  0.]
 [ 1.  1.]
 [ 0.  0.]
 [ 0.  1.]]
[[ 1.  1.]
 [ 1.  0.]
 [ 0.  1.]
 [ 0.  0.]]


In [103]:
print (len(PHIS1))
print(len(PHIS2))
print(len(PHIS3))
print(len(PHIS4))

22
24
24
24


In [104]:
a1 = list(sorted(PHIS1))
a2 = list(sorted(PHIS2))
a3 = list(sorted(PHIS3))
a4 = list(sorted(PHIS4))

In [9]:
network = pyphi.examples.fig16()
state = (1, 0, 0, 1, 1, 1, 0)

In [10]:
condensed = pyphi.compute.condensed(network, state)

In [11]:
condensed

[
 BigMip
 phi: 1.916665
 subsystem: Subsystem((n0, n1, n2))
 cut: Cut (0, 1) --//--> (2,)
 unpartitioned_constellation: 
 
   Concept
   -------
   phi: 0.166667
   mechanism: (0,)
   cause: 
     phi: 0.166667
     purview: (1, 2)
     partition:
       []   0
       -- X -
       1    2
     unpartitioned_repertoire:
       [[[[[[[ 0.        ]]]]
       
       
       
          [[[[ 0.33333333]]]]]
       
       
       
       
         [[[[[ 0.33333333]]]]
       
       
       
          [[[[ 0.33333333]]]]]]]
     partitioned_repertoire:
       [[[[[[[ 0.16666667]]]]
       
       
       
          [[[[ 0.33333333]]]]]
       
       
       
       
         [[[[[ 0.16666667]]]]
       
       
       
          [[[[ 0.33333333]]]]]]]
   effect: 
     phi: 0.25
     purview: (1,)
     partition:
       []   0 
       -- X --
       1    []
     unpartitioned_repertoire:
       [[[[[[[ 0.5]]]]]
       
       
       
       
         [[[[[ 0.5]]]]]]]
     partitioned_repe

In [22]:
ans = pyphi.compute.main_complex(network,state)

In [23]:
ans


BigMip
phi: 1.916665
subsystem: Subsystem((n0, n1, n2))
cut: Cut (0, 1) --//--> (2,)
unpartitioned_constellation: 

  Concept
  -------
  phi: 0.166667
  mechanism: (0,)
  cause: 
    phi: 0.166667
    purview: (1, 2)
    partition:
      []   0
      -- X -
      1    2
    unpartitioned_repertoire:
      [[[[[[[ 0.        ]]]]
      
      
      
         [[[[ 0.33333333]]]]]
      
      
      
      
        [[[[[ 0.33333333]]]]
      
      
      
         [[[[ 0.33333333]]]]]]]
    partitioned_repertoire:
      [[[[[[[ 0.16666667]]]]
      
      
      
         [[[[ 0.33333333]]]]]
      
      
      
      
        [[[[[ 0.16666667]]]]
      
      
      
         [[[[ 0.33333333]]]]]]]
  effect: 
    phi: 0.25
    purview: (1,)
    partition:
      []   0 
      -- X --
      1    []
    unpartitioned_repertoire:
      [[[[[[[ 0.5]]]]]
      
      
      
      
        [[[[[ 0.5]]]]]]]
    partitioned_repertoire:
      [[[[[[[ 0.75]]]]]
      
      
      
      
   

In [26]:
def modeltpm(ka, kb, kc, opt1 = 0, opt2 = 0):
    return np.array([[.25, .25, .25],[1-ka, .25, .25+opt1*.25], [.25, 1-kb, .25+opt1*.25],[1-ka, 1-kb, .25],[.25+opt2*.25, .25+opt2*.25, 1-kc],[1-ka,.25, 1-kc],[.25, 1-kb, 1-kc],[1-ka,1-kb,1-kc]])

In [37]:
tpm = modeltpm(.2,.3,.4, opt1=1, opt2=1)

In [38]:
network = pyphi.Network(tpm)
state = (0,0,0)
subsystem = pyphi.Subsystem(network, state, range(network.size))
phi = pyphi.compute.big_phi(subsystem)
print (phi)

0.148604
